# MLflow Faithfulness Metric Demonstration

This notebook provides a comprehensive guide to using MLflow's Faithfulness metric for evaluating RAG (Retrieval-Augmented Generation) systems and other context-grounded text generation applications.

## What is Faithfulness?

Faithfulness measures whether a generated answer is factually consistent with and supported by the provided context/source documents. It's a critical metric for detecting hallucinations in RAG systems.

### Key Characteristics:
- **Context-grounded**: Evaluates if the answer is supported by the given context
- **Hallucination detection**: Identifies when models generate unsupported information
- **Range**: Typically 0-1, where 1 means fully faithful to context
- **Use cases**: RAG pipelines, document QA, summarization with sources

### Why Faithfulness Matters:
| Problem | Impact | Faithfulness Helps |
|---------|--------|-------------------|
| **Hallucination** | Model invents facts | Detects unsupported claims |
| **Misattribution** | Wrong source cited | Verifies context alignment |
| **Extrapolation** | Goes beyond context | Flags additions |
| **Contradiction** | Conflicts with source | Identifies inconsistencies |

### Faithfulness vs Other Metrics:
| Metric | What it Measures | Best For |
|--------|-----------------|----------|
| **Faithfulness** | Answer supported by context | RAG, grounded generation |
| **Answer Similarity** | Semantic match to reference | QA evaluation |
| **Relevance** | Answer addresses question | Response quality |
| **ROUGE/BLEU** | Text overlap | Summarization, translation |


## 1. Installation and Setup


In [ ]:
# Install required packages
%pip install -q mlflow sentence-transformers pandas numpy matplotlib seaborn plotly scikit-learn transformers torch


In [ ]:
%restart_python


In [ ]:
import mlflow
import mlflow.metrics
from mlflow.metrics import make_metric, MetricValue
import pandas as pd
import numpy as np
import re
from typing import List, Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Sentence embeddings for semantic similarity
from sentence_transformers import SentenceTransformer, CrossEncoder
from sklearn.metrics.pairwise import cosine_similarity

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 100

print(f"MLflow version: {mlflow.__version__}")


In [ ]:
user_email = (
    dbutils.notebook.entry_point.getDbutils()
    .notebook()
    .getContext()
    .tags()
    .get("user")
    .get()
)


## 2. Creating Custom Faithfulness Metrics for MLflow


In [ ]:
class FaithfulnessMetrics:
    """
    Comprehensive faithfulness metrics for MLflow evaluation.
    Measures whether generated answers are supported by the provided context.
    """

    def __init__(self, 
                 embedding_model: str = "all-MiniLM-L6-v2",
                 nli_model: str = "cross-encoder/nli-deberta-v3-small"):
        """
        Initialize with embedding and NLI models.
        
        Args:
            embedding_model: Sentence transformer model for semantic similarity
            nli_model: Cross-encoder model for natural language inference
        """
        print(f"Loading embedding model: {embedding_model}...")
        self.embedding_model = SentenceTransformer(embedding_model)
        self.embedding_model_name = embedding_model
        
        print(f"Loading NLI model: {nli_model}...")
        self.nli_model = CrossEncoder(nli_model)
        self.nli_model_name = nli_model
        
        print("✅ Models loaded successfully!")

    @staticmethod
    def _to_string_list(data):
        """Convert input data to a list of strings."""
        if hasattr(data, 'tolist'):
            return [str(x) for x in data.tolist()]
        if hasattr(data, '__iter__') and not isinstance(data, str):
            return [str(x) for x in data]
        return [str(data)]

    @staticmethod
    def split_into_claims(text: str) -> List[str]:
        """
        Split text into individual claims/sentences for verification.
        
        Args:
            text: The answer text to split
            
        Returns:
            List of individual claims/sentences
        """
        # Split on sentence boundaries
        sentences = re.split(r'(?<=[.!?])\s+', text.strip())
        # Filter out empty strings and very short fragments
        claims = [s.strip() for s in sentences if len(s.strip()) > 10]
        return claims if claims else [text]

    def compute_semantic_faithfulness(self, answer: str, context: str) -> float:
        """
        Compute faithfulness using semantic similarity between answer and context.
        
        Args:
            answer: Generated answer
            context: Source context/document
            
        Returns:
            Faithfulness score between 0 and 1
        """
        # Encode both texts
        answer_embedding = self.embedding_model.encode([answer])[0]
        context_embedding = self.embedding_model.encode([context])[0]
        
        # Compute cosine similarity
        similarity = cosine_similarity([answer_embedding], [context_embedding])[0][0]
        
        # Normalize to 0-1 range
        return max(0, min(1, (similarity + 1) / 2))

    def compute_nli_faithfulness(self, answer: str, context: str) -> Dict[str, float]:
        """
        Compute faithfulness using Natural Language Inference.
        Checks if the context entails the answer.
        
        Args:
            answer: Generated answer
            context: Source context/document
            
        Returns:
            Dictionary with entailment, neutral, and contradiction scores
        """
        # NLI models expect (premise, hypothesis) pairs
        # Here: context is premise, answer is hypothesis
        scores = self.nli_model.predict([(context, answer)])[0]
        
        # Most NLI models return [contradiction, neutral, entailment] or similar
        # The cross-encoder returns a single score for entailment likelihood
        if isinstance(scores, (int, float)):
            # Single score model - treat as entailment score
            entailment_score = 1 / (1 + np.exp(-scores))  # sigmoid
            return {
                "entailment": entailment_score,
                "faithfulness": entailment_score
            }
        else:
            # Multi-class model
            return {
                "contradiction": float(scores[0]),
                "neutral": float(scores[1]),
                "entailment": float(scores[2]),
                "faithfulness": float(scores[2])  # Use entailment as faithfulness
            }

    def compute_claim_faithfulness(self, answer: str, context: str) -> Dict[str, float]:
        """
        Compute faithfulness by verifying individual claims in the answer.
        
        Args:
            answer: Generated answer
            context: Source context/document
            
        Returns:
            Dictionary with per-claim and aggregate faithfulness scores
        """
        claims = self.split_into_claims(answer)
        
        if not claims:
            return {"faithfulness": 1.0, "num_claims": 0, "supported_claims": 0}
        
        claim_scores = []
        for claim in claims:
            # Check if this claim is supported by context using NLI
            nli_result = self.compute_nli_faithfulness(claim, context)
            claim_scores.append(nli_result["faithfulness"])
        
        return {
            "faithfulness": np.mean(claim_scores),
            "min_claim_score": np.min(claim_scores),
            "max_claim_score": np.max(claim_scores),
            "num_claims": len(claims),
            "supported_claims": sum(1 for s in claim_scores if s >= 0.5),
            "claim_scores": claim_scores
        }

    def compute_token_overlap_faithfulness(self, answer: str, context: str) -> float:
        """
        Simple token overlap based faithfulness (baseline).
        
        Args:
            answer: Generated answer
            context: Source context/document
            
        Returns:
            Proportion of answer tokens found in context
        """
        # Tokenize (simple word-based)
        answer_tokens = set(answer.lower().split())
        context_tokens = set(context.lower().split())
        
        # Remove common stop words for better signal
        stop_words = {'the', 'a', 'an', 'is', 'are', 'was', 'were', 'be', 'been', 
                      'being', 'have', 'has', 'had', 'do', 'does', 'did', 'will',
                      'would', 'could', 'should', 'may', 'might', 'must', 'shall',
                      'can', 'need', 'dare', 'ought', 'used', 'to', 'of', 'in',
                      'for', 'on', 'with', 'at', 'by', 'from', 'as', 'into',
                      'through', 'during', 'before', 'after', 'above', 'below',
                      'between', 'under', 'again', 'further', 'then', 'once',
                      'and', 'but', 'or', 'nor', 'so', 'yet', 'both', 'either',
                      'neither', 'not', 'only', 'own', 'same', 'than', 'too',
                      'very', 'just', 'also', 'now', 'it', 'its', 'this', 'that'}
        
        answer_content = answer_tokens - stop_words
        context_content = context_tokens - stop_words
        
        if not answer_content:
            return 1.0  # Empty answer is technically faithful
        
        overlap = answer_content.intersection(context_content)
        return len(overlap) / len(answer_content)

    def create_semantic_faithfulness_metric(self):
        """
        Create MLflow metric for semantic similarity-based faithfulness.
        """
        def semantic_faithfulness_metric(predictions, targets, metrics):
            predictions = self._to_string_list(predictions)
            # For faithfulness, targets should contain the context
            contexts = self._to_string_list(targets)

            scores = [
                self.compute_semantic_faithfulness(pred, ctx)
                for pred, ctx in zip(predictions, contexts)
            ]

            return MetricValue(
                aggregate_results={
                    "semantic_faithfulness_mean": np.mean(scores),
                    "semantic_faithfulness_std": np.std(scores),
                    "semantic_faithfulness_min": np.min(scores),
                    "semantic_faithfulness_max": np.max(scores)
                },
                scores=scores
            )

        return make_metric(
            eval_fn=semantic_faithfulness_metric,
            greater_is_better=True,
            name="semantic_faithfulness"
        )

    def create_nli_faithfulness_metric(self):
        """
        Create MLflow metric for NLI-based faithfulness.
        """
        def nli_faithfulness_metric(predictions, targets, metrics):
            predictions = self._to_string_list(predictions)
            contexts = self._to_string_list(targets)

            scores = []
            entailment_scores = []
            
            for pred, ctx in zip(predictions, contexts):
                result = self.compute_nli_faithfulness(pred, ctx)
                scores.append(result["faithfulness"])
                entailment_scores.append(result.get("entailment", result["faithfulness"]))

            return MetricValue(
                aggregate_results={
                    "nli_faithfulness_mean": np.mean(scores),
                    "nli_faithfulness_std": np.std(scores),
                    "entailment_mean": np.mean(entailment_scores),
                    "high_faithfulness_ratio": sum(1 for s in scores if s >= 0.7) / len(scores)
                },
                scores=scores
            )

        return make_metric(
            eval_fn=nli_faithfulness_metric,
            greater_is_better=True,
            name="nli_faithfulness"
        )

    def create_token_overlap_metric(self):
        """
        Create MLflow metric for token overlap faithfulness (baseline).
        """
        def token_overlap_metric(predictions, targets, metrics):
            predictions = self._to_string_list(predictions)
            contexts = self._to_string_list(targets)

            scores = [
                self.compute_token_overlap_faithfulness(pred, ctx)
                for pred, ctx in zip(predictions, contexts)
            ]

            return MetricValue(
                aggregate_results={
                    "token_overlap_mean": np.mean(scores),
                    "token_overlap_std": np.std(scores),
                    "token_overlap_min": np.min(scores),
                    "token_overlap_max": np.max(scores)
                },
                scores=scores
            )

        return make_metric(
            eval_fn=token_overlap_metric,
            greater_is_better=True,
            name="token_overlap_faithfulness"
        )

    def create_comprehensive_faithfulness_metric(self):
        """
        Create comprehensive metric combining multiple faithfulness approaches.
        """
        def comprehensive_metric(predictions, targets, metrics):
            predictions = self._to_string_list(predictions)
            contexts = self._to_string_list(targets)

            semantic_scores = []
            nli_scores = []
            overlap_scores = []
            combined_scores = []

            for pred, ctx in zip(predictions, contexts):
                sem = self.compute_semantic_faithfulness(pred, ctx)
                nli = self.compute_nli_faithfulness(pred, ctx)["faithfulness"]
                overlap = self.compute_token_overlap_faithfulness(pred, ctx)
                
                semantic_scores.append(sem)
                nli_scores.append(nli)
                overlap_scores.append(overlap)
                
                # Weighted combination (NLI most important, then semantic, then overlap)
                combined = 0.5 * nli + 0.35 * sem + 0.15 * overlap
                combined_scores.append(combined)

            aggregate_results = {
                # Combined score
                "faithfulness_combined": np.mean(combined_scores),
                "faithfulness_std": np.std(combined_scores),
                
                # Individual approaches
                "semantic_faithfulness": np.mean(semantic_scores),
                "nli_faithfulness": np.mean(nli_scores),
                "token_overlap": np.mean(overlap_scores),
                
                # Quality distribution
                "high_faithfulness_count": sum(1 for s in combined_scores if s >= 0.8),
                "medium_faithfulness_count": sum(1 for s in combined_scores if 0.5 <= s < 0.8),
                "low_faithfulness_count": sum(1 for s in combined_scores if s < 0.5),
                
                # Percentages
                "high_faithfulness_pct": sum(1 for s in combined_scores if s >= 0.8) / len(combined_scores) * 100,
                "low_faithfulness_pct": sum(1 for s in combined_scores if s < 0.5) / len(combined_scores) * 100
            }

            return MetricValue(
                aggregate_results=aggregate_results,
                scores=combined_scores
            )

        return make_metric(
            eval_fn=comprehensive_metric,
            greater_is_better=True,
            name="comprehensive_faithfulness"
        )

# Initialize the metrics
faithfulness_metrics = FaithfulnessMetrics()

# Create metric instances
semantic_faithfulness = faithfulness_metrics.create_semantic_faithfulness_metric()
nli_faithfulness = faithfulness_metrics.create_nli_faithfulness_metric()
token_overlap = faithfulness_metrics.create_token_overlap_metric()
comprehensive_faithfulness = faithfulness_metrics.create_comprehensive_faithfulness_metric()

print("\n✅ Custom Faithfulness metrics created successfully!")


In [ ]:
# Create sample RAG data for evaluation
# Each example has: question, context, and different quality answers

rag_data = [
    {
        "question": "What is machine learning?",
        "context": "Machine learning is a subset of artificial intelligence that enables computers to learn from data without being explicitly programmed. It uses algorithms to identify patterns in data and make predictions or decisions.",
        "faithful_answer": "Machine learning is a subset of AI that allows computers to learn from data without explicit programming, using algorithms to find patterns.",
        "partial_faithful": "Machine learning is a type of AI that helps computers learn. It's used in many applications like image recognition.",
        "unfaithful_answer": "Machine learning was invented by Alan Turing in 1950 and requires quantum computers to function properly."
    },
    {
        "question": "What causes climate change?",
        "context": "Climate change is primarily driven by human activities, especially the burning of fossil fuels like coal, oil, and natural gas. These activities release greenhouse gases, particularly carbon dioxide, into the atmosphere, trapping heat and raising global temperatures.",
        "faithful_answer": "Climate change is mainly caused by human activities, particularly burning fossil fuels which release greenhouse gases like carbon dioxide that trap heat.",
        "partial_faithful": "Climate change is caused by greenhouse gases. The temperature is rising globally.",
        "unfaithful_answer": "Climate change is primarily caused by changes in Earth's orbit and volcanic activity, with human impact being minimal."
    },
    {
        "question": "How does photosynthesis work?",
        "context": "Photosynthesis is the process by which plants convert light energy, usually from the sun, into chemical energy stored in glucose. Plants absorb carbon dioxide from the air and water from the soil, using chlorophyll to capture light energy and produce glucose and oxygen.",
        "faithful_answer": "Plants use photosynthesis to convert light energy into glucose by absorbing CO2 and water, using chlorophyll to capture light and producing glucose and oxygen.",
        "partial_faithful": "Photosynthesis converts light into food for plants. Plants need sunlight to grow.",
        "unfaithful_answer": "Photosynthesis is how plants breathe oxygen and release carbon dioxide at night through their roots."
    },
    {
        "question": "What is the capital of France?",
        "context": "France is a country in Western Europe. Its capital city is Paris, which is also its largest city with a population of about 2.2 million in the city proper. Paris is known for the Eiffel Tower and the Louvre Museum.",
        "faithful_answer": "The capital of France is Paris, which is also its largest city with about 2.2 million people, known for the Eiffel Tower and Louvre Museum.",
        "partial_faithful": "Paris is the capital of France. It's a beautiful city.",
        "unfaithful_answer": "The capital of France is Lyon, which became the capital in 2010 after a national referendum."
    },
    {
        "question": "How does the immune system work?",
        "context": "The immune system protects the body against pathogens like bacteria and viruses. It consists of white blood cells, antibodies, and other components. When a pathogen enters the body, the immune system recognizes it as foreign and mounts a response to eliminate it.",
        "faithful_answer": "The immune system protects against pathogens using white blood cells and antibodies. It recognizes foreign invaders and eliminates them.",
        "partial_faithful": "The immune system fights diseases. White blood cells are important for health.",
        "unfaithful_answer": "The immune system is located primarily in the brain and strengthens automatically with age without any external factors."
    },
    {
        "question": "What is DNA?",
        "context": "DNA (deoxyribonucleic acid) is a molecule that carries genetic instructions for living organisms. It has a double helix structure discovered by Watson and Crick. DNA contains four bases: adenine, thymine, guanine, and cytosine.",
        "faithful_answer": "DNA is a molecule carrying genetic instructions, with a double helix structure. It contains four bases: adenine, thymine, guanine, and cytosine.",
        "partial_faithful": "DNA contains genetic information. It's found in all living things.",
        "unfaithful_answer": "DNA is a type of protein that was discovered in 1990. It has a single helix structure and contains only two bases."
    }
]

# Convert to DataFrame
df_rag = pd.DataFrame(rag_data)

# Calculate faithfulness scores for initial analysis
print("📊 Computing initial faithfulness scores...")

for answer_type in ['faithful_answer', 'partial_faithful', 'unfaithful_answer']:
    df_rag[f'{answer_type}_semantic'] = [
        faithfulness_metrics.compute_semantic_faithfulness(row[answer_type], row['context'])
        for _, row in df_rag.iterrows()
    ]
    df_rag[f'{answer_type}_nli'] = [
        faithfulness_metrics.compute_nli_faithfulness(row[answer_type], row['context'])['faithfulness']
        for _, row in df_rag.iterrows()
    ]
    df_rag[f'{answer_type}_overlap'] = [
        faithfulness_metrics.compute_token_overlap_faithfulness(row[answer_type], row['context'])
        for _, row in df_rag.iterrows()
    ]

print(f"\n📊 Created {len(df_rag)} RAG examples")
print("\n" + "="*100)
print("Sample Data Preview:")
print("="*100)

for idx, row in df_rag.head(3).iterrows():
    print(f"\n{idx+1}. Question: {row['question']}")
    print(f"   Context: {row['context'][:80]}...")
    print(f"\n   Faithful Answer: {row['faithful_answer'][:60]}...")
    print(f"   → Semantic: {row['faithful_answer_semantic']:.3f}, NLI: {row['faithful_answer_nli']:.3f}")
    print(f"\n   Partial Faithful: {row['partial_faithful'][:60]}...")
    print(f"   → Semantic: {row['partial_faithful_semantic']:.3f}, NLI: {row['partial_faithful_nli']:.3f}")
    print(f"\n   Unfaithful: {row['unfaithful_answer'][:60]}...")
    print(f"   → Semantic: {row['unfaithful_answer_semantic']:.3f}, NLI: {row['unfaithful_answer_nli']:.3f}")


## 5. Comprehensive Results Analysis


In [ ]:
# Create comparison table
comparison_data = []

for model_name, results in evaluation_results.items():
    detailed = results["detailed"]
    row = {
        "Model": model_name,
        "Faithfulness (Combined)": detailed["faithfulness_mean"],
        "Semantic": detailed["semantic_mean"],
        "NLI": detailed["nli_mean"],
        "Token Overlap": detailed["overlap_mean"],
        "High Quality": int(detailed["high_faithfulness"]),
        "Low Quality": int(detailed["low_faithfulness"]),
        "Hallucinations": int(detailed["potential_hallucinations"])
    }
    comparison_data.append(row)

comparison_df = pd.DataFrame(comparison_data)

print("\n📊 Model Comparison Results:")
print("="*120)

# Format percentages for display
display_df = comparison_df.copy()
for col in ["Faithfulness (Combined)", "Semantic", "NLI", "Token Overlap"]:
    display_df[col] = display_df[col].apply(lambda x: f"{x*100:.1f}%")

print(display_df.to_string(index=False))

# Detailed analysis
print("\n\n📈 Detailed Faithfulness Analysis:")
print("="*120)

for model_name, results in evaluation_results.items():
    detailed = results["detailed"]
    print(f"\n{model_name}:")
    print(f"  Combined Faithfulness: {detailed['faithfulness_mean']*100:.1f}% (σ={detailed['faithfulness_std']*100:.1f}%)")
    print(f"  Range: [{detailed['faithfulness_min']*100:.1f}%, {detailed['faithfulness_max']*100:.1f}%]")
    print(f"  Quality Distribution: High={detailed['high_faithfulness']}, Medium={detailed['medium_faithfulness']}, Low={detailed['low_faithfulness']}")
    print(f"  Potential Hallucinations: {detailed['potential_hallucinations']}")


## 6. Advanced Visualizations


In [ ]:
# Create comprehensive visualizations with Plotly
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "Faithfulness Scores by Model",
        "Metric Comparison (Semantic vs NLI vs Overlap)",
        "Per-Question Faithfulness Heatmap",
        "Quality Distribution"
    ),
    specs=[
        [{"type": "bar"}, {"type": "bar"}],
        [{"type": "heatmap"}, {"type": "pie"}]
    ]
)

colors = {
    "High-Faithfulness-RAG": "#2ecc71",
    "Medium-Faithfulness-RAG": "#f39c12",
    "Low-Faithfulness-Baseline": "#e74c3c"
}

# Plot 1: Overall Faithfulness Scores
model_names = list(evaluation_results.keys())
faithfulness_scores = [evaluation_results[m]["detailed"]["faithfulness_mean"] * 100 for m in model_names]

fig.add_trace(
    go.Bar(
        name="Combined Faithfulness",
        x=[m.replace("-", " ") for m in model_names],
        y=faithfulness_scores,
        marker_color=[colors[m] for m in model_names],
        text=[f"{s:.1f}%" for s in faithfulness_scores],
        textposition='outside'
    ),
    row=1, col=1
)

# Plot 2: Metric Comparison (grouped bar)
metrics_to_plot = ["semantic_mean", "nli_mean", "overlap_mean"]
metric_labels = ["Semantic", "NLI", "Token Overlap"]
bar_colors = ["#3498db", "#9b59b6", "#1abc9c"]

for i, (metric, label, color) in enumerate(zip(metrics_to_plot, metric_labels, bar_colors)):
    values = [evaluation_results[m]["detailed"][metric] * 100 for m in model_names]
    fig.add_trace(
        go.Bar(
            name=label,
            x=[m.replace("-", " ") for m in model_names],
            y=values,
            marker_color=color,
            text=[f"{v:.1f}%" for v in values],
            textposition='outside'
        ),
        row=1, col=2
    )

# Plot 3: Heatmap of per-question faithfulness
heatmap_data = []
questions_short = [q[:30] + "..." for q in df_rag["question"].tolist()]

for answer_type, model_name in [
    ("faithful_answer", "High"),
    ("partial_faithful", "Medium"),
    ("unfaithful_answer", "Low")
]:
    scores = []
    for _, row in df_rag.iterrows():
        combined = (
            0.5 * row[f'{answer_type}_nli'] +
            0.35 * row[f'{answer_type}_semantic'] +
            0.15 * row[f'{answer_type}_overlap']
        )
        scores.append(combined)
    heatmap_data.append(scores)

fig.add_trace(
    go.Heatmap(
        z=heatmap_data,
        x=[f"Q{i+1}" for i in range(len(df_rag))],
        y=["High", "Medium", "Low"],
        colorscale=[[0, "#e74c3c"], [0.5, "#f39c12"], [1, "#2ecc71"]],
        text=np.round(np.array(heatmap_data), 2),
        texttemplate="%{text}",
        textfont={"size": 10},
        zmin=0,
        zmax=1
    ),
    row=2, col=1
)

# Plot 4: Pie chart showing quality distribution for all answers combined
all_combined = []
for answer_type in ['faithful_answer', 'partial_faithful', 'unfaithful_answer']:
    for _, row in df_rag.iterrows():
        combined = (
            0.5 * row[f'{answer_type}_nli'] +
            0.35 * row[f'{answer_type}_semantic'] +
            0.15 * row[f'{answer_type}_overlap']
        )
        all_combined.append(combined)

high_count = sum(1 for s in all_combined if s >= 0.8)
medium_count = sum(1 for s in all_combined if 0.5 <= s < 0.8)
low_count = sum(1 for s in all_combined if s < 0.5)

fig.add_trace(
    go.Pie(
        labels=["High Faithfulness", "Medium Faithfulness", "Low Faithfulness"],
        values=[high_count, medium_count, low_count],
        marker_colors=["#2ecc71", "#f39c12", "#e74c3c"],
        title="Overall Distribution"
    ),
    row=2, col=2
)

# Update layout
fig.update_layout(
    title_text="Comprehensive Faithfulness Analysis",
    showlegend=True,
    height=800,
    width=1200,
    barmode='group'
)

fig.update_yaxes(title_text="Faithfulness (%)", row=1, col=1)
fig.update_yaxes(title_text="Score (%)", row=1, col=2)
fig.update_xaxes(title_text="Question", row=2, col=1)

fig.show()


In [ ]:
# Additional visualization: Radar chart comparing models
fig2 = go.Figure()

categories = ['Semantic', 'NLI', 'Token Overlap', 'Combined', 'High Quality %']

for model_name in model_names:
    detailed = evaluation_results[model_name]["detailed"]
    values = [
        detailed["semantic_mean"] * 100,
        detailed["nli_mean"] * 100,
        detailed["overlap_mean"] * 100,
        detailed["faithfulness_mean"] * 100,
        detailed["high_pct"]
    ]
    
    fig2.add_trace(go.Scatterpolar(
        r=values + [values[0]],  # Close the loop
        theta=categories + [categories[0]],
        name=model_name.replace("-", " "),
        line=dict(color=colors[model_name])
    ))

fig2.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 100]
        )
    ),
    title="Model Comparison Radar Chart",
    showlegend=True,
    height=500,
    width=700
)

fig2.show()


In [ ]:
def analyze_hallucinations(predictions: List[str],
                           contexts: List[str],
                           questions: List[str] = None,
                           threshold: float = 0.5) -> pd.DataFrame:
    """
    Analyze potential hallucinations in model outputs.
    
    Args:
        predictions: List of model predictions
        contexts: List of source contexts
        questions: Optional list of questions
        threshold: Faithfulness threshold below which to flag as potential hallucination
        
    Returns:
        DataFrame with hallucination analysis
    """
    analysis_data = []
    
    for i, (pred, ctx) in enumerate(zip(predictions, contexts)):
        # Calculate all faithfulness scores
        semantic = faithfulness_metrics.compute_semantic_faithfulness(pred, ctx)
        nli_result = faithfulness_metrics.compute_nli_faithfulness(pred, ctx)
        overlap = faithfulness_metrics.compute_token_overlap_faithfulness(pred, ctx)
        
        # Combined score
        combined = 0.5 * nli_result["faithfulness"] + 0.35 * semantic + 0.15 * overlap
        
        # Determine hallucination category
        if combined >= 0.8:
            category = "Highly Faithful"
            risk = "Low"
        elif combined >= 0.6:
            category = "Mostly Faithful"
            risk = "Low"
        elif combined >= 0.4:
            category = "Partially Faithful"
            risk = "Medium"
        elif combined >= 0.2:
            category = "Likely Hallucination"
            risk = "High"
        else:
            category = "Severe Hallucination"
            risk = "Critical"
        
        # Analyze specific hallucination indicators
        indicators = []
        if nli_result["faithfulness"] < 0.3:
            indicators.append("NLI contradiction")
        if semantic < 0.5:
            indicators.append("Low semantic similarity")
        if overlap < 0.3:
            indicators.append("Low token overlap")
        
        row = {
            "idx": i,
            "question": questions[i] if questions else f"Q{i+1}",
            "prediction": pred[:100] + "..." if len(pred) > 100 else pred,
            "context_preview": ctx[:80] + "..." if len(ctx) > 80 else ctx,
            "semantic_score": semantic,
            "nli_score": nli_result["faithfulness"],
            "overlap_score": overlap,
            "combined_score": combined,
            "category": category,
            "risk_level": risk,
            "indicators": ", ".join(indicators) if indicators else "None"
        }
        analysis_data.append(row)
    
    return pd.DataFrame(analysis_data)

# Analyze all three answer types
print("🔍 Hallucination Analysis:")
print("="*100)

for answer_type, label in [
    ("unfaithful_answer", "Low-Faithfulness-Baseline"),
    ("partial_faithful", "Medium-Faithfulness-RAG"),
    ("faithful_answer", "High-Faithfulness-RAG")
]:
    analysis = analyze_hallucinations(
        df_rag[answer_type].tolist(),
        df_rag["context"].tolist(),
        df_rag["question"].tolist()
    )
    
    print(f"\n📌 {label}:")
    
    # Category distribution
    category_counts = analysis["category"].value_counts()
    for category, count in category_counts.items():
        pct = count / len(analysis) * 100
        print(f"   {category}: {count} ({pct:.1f}%)")
    
    # Show examples with risk
    high_risk = analysis[analysis["risk_level"].isin(["High", "Critical"])]
    if len(high_risk) > 0:
        print(f"\n   ⚠️  High-Risk Examples ({len(high_risk)}):")
        for _, row in high_risk.head(2).iterrows():
            print(f"      Q: {row['question'][:50]}...")
            print(f"      Answer: {row['prediction'][:60]}...")
            print(f"      Score: {row['combined_score']:.2f} | Indicators: {row['indicators']}")


## 8. Production-Ready Evaluation Pipeline


In [ ]:
class FaithfulnessPipeline:
    """
    Production-ready faithfulness evaluation pipeline for MLflow.
    Includes comprehensive metrics, hallucination detection, and reporting.
    """
    
    def __init__(self,
                 experiment_name: str,
                 tracking_uri: str = None,
                 embedding_model: str = "all-MiniLM-L6-v2",
                 nli_model: str = "cross-encoder/nli-deberta-v3-small"):
        """
        Initialize the faithfulness pipeline.
        
        Args:
            experiment_name: MLflow experiment name
            tracking_uri: Optional MLflow tracking URI
            embedding_model: Sentence transformer model
            nli_model: NLI cross-encoder model
        """
        self.experiment_name = experiment_name
        
        if tracking_uri:
            mlflow.set_tracking_uri(tracking_uri)
        
        mlflow.set_experiment(experiment_name)
        
        # Initialize metrics
        self.metrics_calculator = FaithfulnessMetrics(embedding_model, nli_model)
        
        # Create metric suite
        self.metrics = [
            self.metrics_calculator.create_semantic_faithfulness_metric(),
            self.metrics_calculator.create_nli_faithfulness_metric(),
            self.metrics_calculator.create_token_overlap_metric(),
            self.metrics_calculator.create_comprehensive_faithfulness_metric()
        ]
    
    def evaluate_model(self,
                       model_name: str,
                       predictions: List[str],
                       contexts: List[str],
                       questions: List[str] = None,
                       metadata: Dict = None) -> Dict:
        """
        Evaluate a model and log results to MLflow.
        
        Args:
            model_name: Name of the model being evaluated
            predictions: Model predictions/answers
            contexts: Source contexts
            questions: Optional questions for context
            metadata: Optional metadata to log
            
        Returns:
            Dictionary with evaluation results
        """
        with mlflow.start_run(run_name=model_name):
            # Log parameters
            mlflow.log_param("model_name", model_name)
            mlflow.log_param("num_samples", len(predictions))
            mlflow.log_param("pipeline_version", "1.0.0")
            mlflow.log_param("embedding_model", self.metrics_calculator.embedding_model_name)
            mlflow.log_param("nli_model", self.metrics_calculator.nli_model_name)
            
            if metadata:
                for key, value in metadata.items():
                    mlflow.log_param(key, value)
            
            # Prepare evaluation dataframe
            eval_df = pd.DataFrame({
                "predictions": predictions,
                "targets": contexts
            })
            
            # Run MLflow evaluation
            results = mlflow.evaluate(
                data=eval_df,
                targets="targets",
                predictions="predictions",
                extra_metrics=self.metrics,
                evaluators="default"
            )
            
            # Calculate detailed metrics
            detailed_metrics = self._calculate_all_metrics(predictions, contexts)
            
            # Log detailed metrics
            for key, value in detailed_metrics.items():
                if isinstance(value, (int, float)):
                    mlflow.log_metric(key, value)
            
            # Run hallucination analysis
            hallucination_analysis = analyze_hallucinations(predictions, contexts, questions)
            
            # Log hallucination metrics
            risk_counts = hallucination_analysis["risk_level"].value_counts()
            for risk, count in risk_counts.items():
                safe_name = risk.lower().replace(" ", "_")
                mlflow.log_metric(f"risk_{safe_name}_count", count)
            
            # Generate and log report
            report = self._generate_evaluation_report(
                model_name, detailed_metrics, hallucination_analysis
            )
            mlflow.log_text(report, "faithfulness_report.txt")
            
            # Generate recommendations
            recommendations = self._generate_recommendations(detailed_metrics)
            mlflow.log_text(recommendations, "recommendations.txt")
            
            return {
                "metrics": results.metrics,
                "detailed": detailed_metrics,
                "hallucination_analysis": hallucination_analysis,
                "report": report,
                "recommendations": recommendations
            }
    
    def _calculate_all_metrics(self,
                               predictions: List[str],
                               contexts: List[str]) -> Dict:
        """Calculate comprehensive metrics."""
        semantic_scores = []
        nli_scores = []
        overlap_scores = []
        
        for pred, ctx in zip(predictions, contexts):
            semantic_scores.append(self.metrics_calculator.compute_semantic_faithfulness(pred, ctx))
            nli_scores.append(self.metrics_calculator.compute_nli_faithfulness(pred, ctx)['faithfulness'])
            overlap_scores.append(self.metrics_calculator.compute_token_overlap_faithfulness(pred, ctx))
        
        combined_scores = [0.5*n + 0.35*s + 0.15*o for n, s, o in zip(nli_scores, semantic_scores, overlap_scores)]
        
        return {
            # Combined metrics
            "faithfulness_combined": np.mean(combined_scores),
            "faithfulness_std": np.std(combined_scores),
            "faithfulness_min": np.min(combined_scores),
            "faithfulness_max": np.max(combined_scores),
            "faithfulness_median": np.median(combined_scores),
            
            # Individual metrics
            "semantic_mean": np.mean(semantic_scores),
            "nli_mean": np.mean(nli_scores),
            "overlap_mean": np.mean(overlap_scores),
            
            # Quality distribution
            "high_faithfulness": sum(1 for s in combined_scores if s >= 0.8),
            "medium_faithfulness": sum(1 for s in combined_scores if 0.5 <= s < 0.8),
            "low_faithfulness": sum(1 for s in combined_scores if s < 0.5),
            
            # Percentages
            "high_pct": sum(1 for s in combined_scores if s >= 0.8) / len(combined_scores) * 100,
            "low_pct": sum(1 for s in combined_scores if s < 0.5) / len(combined_scores) * 100,
            
            # Hallucination indicators
            "potential_hallucinations": sum(1 for s in combined_scores if s < 0.4),
            "severe_hallucinations": sum(1 for s in combined_scores if s < 0.2),
            
            # Quality indicators
            "acceptable_quality": 1.0 if np.mean(combined_scores) >= 0.6 else 0.0,
            "high_quality": 1.0 if np.mean(combined_scores) >= 0.8 else 0.0
        }
    
    def _generate_evaluation_report(self,
                                    model_name: str,
                                    metrics: Dict,
                                    hallucination_analysis: pd.DataFrame) -> str:
        """Generate a text report."""
        report_lines = [
            "="*80,
            f"FAITHFULNESS EVALUATION REPORT: {model_name}",
            "="*80,
            "",
            "PERFORMANCE METRICS:",
            "-"*40,
            f"  Combined Faithfulness:    {metrics['faithfulness_combined']*100:.1f}%",
            f"  Semantic Faithfulness:    {metrics['semantic_mean']*100:.1f}%",
            f"  NLI Faithfulness:         {metrics['nli_mean']*100:.1f}%",
            f"  Token Overlap:            {metrics['overlap_mean']*100:.1f}%",
            "",
            f"  Score Range: [{metrics['faithfulness_min']*100:.1f}%, {metrics['faithfulness_max']*100:.1f}%]",
            f"  Standard Deviation: {metrics['faithfulness_std']*100:.1f}%",
            "",
            "QUALITY DISTRIBUTION:",
            "-"*40,
            f"  High Faithfulness (>=80%):    {int(metrics['high_faithfulness'])} ({metrics['high_pct']:.1f}%)",
            f"  Medium Faithfulness (50-80%): {int(metrics['medium_faithfulness'])} ({100-metrics['high_pct']-metrics['low_pct']:.1f}%)",
            f"  Low Faithfulness (<50%):      {int(metrics['low_faithfulness'])} ({metrics['low_pct']:.1f}%)",
            "",
            "HALLUCINATION ANALYSIS:",
            "-"*40,
            f"  Potential Hallucinations:  {int(metrics['potential_hallucinations'])}",
            f"  Severe Hallucinations:     {int(metrics['severe_hallucinations'])}",
            ""
        ]
        
        # Add risk distribution
        risk_counts = hallucination_analysis["risk_level"].value_counts()
        report_lines.append("RISK DISTRIBUTION:")
        report_lines.append("-"*40)
        for risk in ["Low", "Medium", "High", "Critical"]:
            count = risk_counts.get(risk, 0)
            pct = count / len(hallucination_analysis) * 100
            report_lines.append(f"  {risk}: {count} ({pct:.1f}%)")
        
        report_lines.extend([
            "",
            "QUALITY ASSESSMENT:",
            "-"*40,
            f"  Acceptable Quality (>=60%): {'Yes' if metrics['acceptable_quality'] else 'No'}",
            f"  High Quality (>=80%):       {'Yes' if metrics['high_quality'] else 'No'}",
            "",
            "="*80
        ])
        
        return "\n".join(report_lines)
    
    def _generate_recommendations(self, metrics: Dict) -> str:
        """Generate actionable recommendations."""
        recommendations = ["RECOMMENDATIONS:", "="*50, ""]
        
        faithfulness = metrics['faithfulness_combined']
        
        if faithfulness < 0.4:
            recommendations.extend([
                "⚠️ CRITICAL: Very low faithfulness (<40%)",
                "",
                "Immediate actions required:",
                "1. Review and improve retrieval pipeline",
                "2. Add explicit grounding instructions to prompts",
                "3. Implement citation requirements",
                "4. Consider using more restrictive generation parameters",
                "5. Add post-generation fact-checking",
                ""
            ])
        elif faithfulness < 0.6:
            recommendations.extend([
                "⚠️ WARNING: Below acceptable threshold (<60%)",
                "",
                "Actions to consider:",
                "1. Improve retriever relevance scoring",
                "2. Add chain-of-thought prompting for grounding",
                "3. Implement answer verification step",
                "4. Review failure cases for patterns",
                ""
            ])
        elif faithfulness < 0.8:
            recommendations.extend([
                "✓ ACCEPTABLE: Good performance (60-80%)",
                "",
                "Potential improvements:",
                "1. Focus on edge cases with low scores",
                "2. Fine-tune for specific failure patterns",
                "3. Add confidence scoring for uncertain answers",
                ""
            ])
        else:
            recommendations.extend([
                "✅ EXCELLENT: High faithfulness (>=80%)",
                "",
                "Maintenance considerations:",
                "1. Monitor for drift in production",
                "2. Expand test set for edge cases",
                "3. Consider stricter thresholds",
                ""
            ])
        
        # Add hallucination-specific recommendations
        if metrics['potential_hallucinations'] > 0:
            recommendations.extend([
                f"🔍 Hallucination Alert: {int(metrics['potential_hallucinations'])} potential hallucinations detected",
                "",
                "Hallucination mitigation:",
                "1. Implement answer-context alignment checks",
                "2. Add explicit 'I don't know' handling",
                "3. Use constrained decoding when possible",
                "4. Add human-in-the-loop for critical answers",
                ""
            ])
        
        return "\n".join(recommendations)
    
    def compare_models(self,
                       model_results: Dict[str, Dict],
                       baseline_name: str = None) -> pd.DataFrame:
        """
        Compare multiple model results.
        
        Args:
            model_results: Dictionary mapping model names to their results
            baseline_name: Optional baseline model name for comparison
            
        Returns:
            DataFrame with comparison results
        """
        comparison_data = []
        baseline_score = None
        
        if baseline_name and baseline_name in model_results:
            baseline_score = model_results[baseline_name]["detailed"]["faithfulness_combined"]
        
        for model_name, results in model_results.items():
            detailed = results["detailed"]
            
            row = {
                "Model": model_name,
                "Faithfulness": detailed["faithfulness_combined"],
                "Semantic": detailed["semantic_mean"],
                "NLI": detailed["nli_mean"],
                "Overlap": detailed["overlap_mean"],
                "High Quality %": detailed["high_pct"],
                "Hallucinations": int(detailed["potential_hallucinations"])
            }
            
            if baseline_score is not None:
                row["vs Baseline"] = detailed["faithfulness_combined"] - baseline_score
            
            comparison_data.append(row)
        
        return pd.DataFrame(comparison_data).sort_values("Faithfulness", ascending=False)

# Demo the production pipeline
print("🚀 Production Pipeline Demo")
print("="*100)

pipeline = FaithfulnessPipeline(
    experiment_name=f"/Users/{user_email}/faithfulness-production-demo"
)

# Evaluate all models through the pipeline
pipeline_results = {}

for model_name, config in models_config.items():
    print(f"\n📝 Evaluating {model_name}...")
    result = pipeline.evaluate_model(
        model_name,
        config["predictions"],
        df_rag["context"].tolist(),
        df_rag["question"].tolist(),
        config["config"]
    )
    pipeline_results[model_name] = result

print("\n✅ Pipeline evaluation completed!")


In [ ]:
# Show model comparison
comparison = pipeline.compare_models(pipeline_results, baseline_name="Low-Faithfulness-Baseline")

print("\n📊 Model Comparison (vs Low-Faithfulness-Baseline):")
print("="*120)

# Format for display
display_comparison = comparison.copy()
for col in ["Faithfulness", "Semantic", "NLI", "Overlap"]:
    display_comparison[col] = display_comparison[col].apply(lambda x: f"{x*100:.1f}%")
if "vs Baseline" in display_comparison.columns:
    display_comparison["vs Baseline"] = display_comparison["vs Baseline"].apply(lambda x: f"{x*100:+.1f}%")
display_comparison["High Quality %"] = display_comparison["High Quality %"].apply(lambda x: f"{x:.1f}%")

print(display_comparison.to_string(index=False))


In [ ]:
# Display reports and recommendations
for model_name, result in pipeline_results.items():
    print(f"\n{'='*100}")
    print(result["report"])
    print("\n" + result["recommendations"])


## 9. Advanced Use Cases


In [ ]:
# Advanced Use Case 1: Claim-level faithfulness with verification
def create_claim_level_metric(metrics_calculator):
    """
    Create metric that evaluates each claim in the answer separately.
    Useful for long-form answers with multiple statements.
    """
    def claim_level_metric(predictions, targets, metrics):
        predictions = FaithfulnessMetrics._to_string_list(predictions)
        contexts = FaithfulnessMetrics._to_string_list(targets)
        
        all_scores = []
        claim_counts = []
        supported_counts = []
        
        for pred, ctx in zip(predictions, contexts):
            result = metrics_calculator.compute_claim_faithfulness(pred, ctx)
            all_scores.append(result["faithfulness"])
            claim_counts.append(result["num_claims"])
            supported_counts.append(result["supported_claims"])
        
        return MetricValue(
            aggregate_results={
                "claim_faithfulness_mean": np.mean(all_scores),
                "total_claims": sum(claim_counts),
                "supported_claims": sum(supported_counts),
                "claim_support_ratio": sum(supported_counts) / max(sum(claim_counts), 1)
            },
            scores=all_scores
        )
    
    return make_metric(
        eval_fn=claim_level_metric,
        greater_is_better=True,
        name="claim_level_faithfulness"
    )

# Demo claim-level analysis
print("📊 Claim-Level Faithfulness Analysis:")
print("="*100)

for idx, row in df_rag.head(3).iterrows():
    print(f"\nQuestion: {row['question']}")
    print(f"Context: {row['context'][:80]}...")
    
    for answer_type, label in [("faithful_answer", "Faithful"), ("unfaithful_answer", "Unfaithful")]:
        result = faithfulness_metrics.compute_claim_faithfulness(row[answer_type], row["context"])
        print(f"\n  {label} Answer: {row[answer_type][:60]}...")
        print(f"  Claims found: {result['num_claims']}, Supported: {result['supported_claims']}")
        print(f"  Overall faithfulness: {result['faithfulness']:.3f}")


In [ ]:
# Advanced Use Case 2: Multi-context faithfulness (multiple retrieved documents)
def create_multi_context_metric(metrics_calculator, aggregation: str = "max"):
    """
    Create metric for answers grounded in multiple retrieved documents.
    Uses max/mean/any aggregation across contexts.
    
    Args:
        metrics_calculator: FaithfulnessMetrics instance
        aggregation: How to aggregate scores across contexts ("max", "mean", "any")
    """
    def multi_context_metric(predictions, targets, metrics):
        predictions = FaithfulnessMetrics._to_string_list(predictions)
        # Targets expected to be pipe-separated multiple contexts
        multi_contexts = FaithfulnessMetrics._to_string_list(targets)
        
        final_scores = []
        
        for pred, ctx_string in zip(predictions, multi_contexts):
            # Split multiple contexts
            contexts = [c.strip() for c in ctx_string.split('|||')]
            
            # Score against each context
            context_scores = []
            for ctx in contexts:
                if ctx:
                    nli = metrics_calculator.compute_nli_faithfulness(pred, ctx)['faithfulness']
                    context_scores.append(nli)
            
            # Aggregate scores
            if not context_scores:
                final_scores.append(0.0)
            elif aggregation == "max":
                final_scores.append(max(context_scores))
            elif aggregation == "mean":
                final_scores.append(np.mean(context_scores))
            elif aggregation == "any":
                final_scores.append(1.0 if any(s >= 0.5 for s in context_scores) else 0.0)
            else:
                final_scores.append(max(context_scores))
        
        return MetricValue(
            aggregate_results={
                f"multi_context_faithfulness_{aggregation}": np.mean(final_scores),
                "num_with_support": sum(1 for s in final_scores if s >= 0.5)
            },
            scores=final_scores
        )
    
    return make_metric(
        eval_fn=multi_context_metric,
        greater_is_better=True,
        name=f"multi_context_faithfulness_{aggregation}"
    )

# Demo with multi-context data
multi_context_data = pd.DataFrame({
    "question": [
        "What are the health benefits of green tea?",
        "How does solar energy work?"
    ],
    "contexts": [
        "Green tea contains antioxidants called catechins. ||| Green tea may help reduce the risk of heart disease. ||| Studies show green tea can boost metabolism.",
        "Solar panels convert sunlight into electricity. ||| Photovoltaic cells use semiconductors to generate power. ||| Solar energy is renewable and sustainable."
    ],
    "answer": [
        "Green tea is rich in antioxidants and may help reduce heart disease risk while boosting metabolism.",
        "Solar panels use photovoltaic cells with semiconductors to convert sunlight into renewable electricity."
    ]
})

print("\n📊 Multi-Context Faithfulness Demo:")
print("="*100)

for idx, row in multi_context_data.iterrows():
    print(f"\nQuestion: {row['question']}")
    print(f"Answer: {row['answer']}")
    print(f"Contexts: {len(row['contexts'].split('|||'))} retrieved documents")
    
    # Score against each context
    contexts = row['contexts'].split('|||')
    for i, ctx in enumerate(contexts):
        score = faithfulness_metrics.compute_nli_faithfulness(row['answer'], ctx.strip())['faithfulness']
        print(f"  Context {i+1}: {score:.3f}")


In [ ]:
# Advanced Use Case 3: Citation verification
def create_citation_faithfulness_metric(metrics_calculator):
    """
    Create metric that verifies cited claims match their sources.
    Expects answers with inline citations like [1], [2], etc.
    """
    def citation_metric(predictions, targets, metrics):
        predictions = FaithfulnessMetrics._to_string_list(predictions)
        # Targets contain numbered sources separated by |||
        source_lists = FaithfulnessMetrics._to_string_list(targets)
        
        scores = []
        citation_accuracy_list = []
        
        for pred, sources_str in zip(predictions, source_lists):
            # Parse sources
            sources = {i+1: s.strip() for i, s in enumerate(sources_str.split('|||'))}
            
            # Find citations in answer
            citation_pattern = r'\[(\d+)\]'
            citations = re.findall(citation_pattern, pred)
            
            if not citations:
                # No citations - evaluate overall
                all_sources = ' '.join(sources.values())
                nli = metrics_calculator.compute_nli_faithfulness(pred, all_sources)['faithfulness']
                scores.append(nli)
                citation_accuracy_list.append(1.0)  # No citations to verify
            else:
                # Verify each citation
                citation_scores = []
                for cite_num in citations:
                    cite_num = int(cite_num)
                    if cite_num in sources:
                        # Find the claim before/around this citation
                        nli = metrics_calculator.compute_nli_faithfulness(pred, sources[cite_num])['faithfulness']
                        citation_scores.append(nli)
                
                if citation_scores:
                    scores.append(np.mean(citation_scores))
                    citation_accuracy_list.append(sum(1 for s in citation_scores if s >= 0.5) / len(citation_scores))
                else:
                    scores.append(0.0)
                    citation_accuracy_list.append(0.0)
        
        return MetricValue(
            aggregate_results={
                "citation_faithfulness": np.mean(scores),
                "citation_accuracy": np.mean(citation_accuracy_list)
            },
            scores=scores
        )
    
    return make_metric(
        eval_fn=citation_metric,
        greater_is_better=True,
        name="citation_faithfulness"
    )

# Demo citation verification
citation_data = pd.DataFrame({
    "question": ["What is the capital of France?"],
    "sources": ["France is a country in Western Europe. Its capital is Paris. ||| Paris has a population of about 2.2 million. ||| The Eiffel Tower is located in Paris."],
    "answer": ["The capital of France is Paris [1], with a population of about 2.2 million people [2]. The city is famous for the Eiffel Tower [3]."]
})

print("\n📊 Citation Verification Demo:")
print("="*100)

for idx, row in citation_data.iterrows():
    print(f"\nQuestion: {row['question']}")
    print(f"Answer: {row['answer']}")
    sources = row['sources'].split('|||')
    print("Sources:")
    for i, src in enumerate(sources):
        print(f"  [{i+1}] {src.strip()[:60]}...")


## 10. Best Practices and Guidelines


In [ ]:
# Best practices reference
best_practices = """
╔══════════════════════════════════════════════════════════════════════════════════╗
║                      FAITHFULNESS METRIC BEST PRACTICES                          ║
╚══════════════════════════════════════════════════════════════════════════════════╝

┌─────────────────────────────────────────────────────────────────────────────────┐
│ 1. CHOOSING THE RIGHT FAITHFULNESS APPROACH                                      │
├─────────────────────────────────────────────────────────────────────────────────┤
│                                                                                  │
│  Use Case                      │ Recommended Approach                            │
│  ────────────────────────────────────────────────────────────────────────────── │
│  RAG QA Systems                │ NLI + Semantic (combined)                       │
│  Document Summarization        │ Claim-level verification                        │
│  Fact Verification             │ NLI-based entailment                            │
│  Multi-document QA             │ Multi-context faithfulness                      │
│  Cited Answers                 │ Citation verification                           │
│  Quick Baseline                │ Token overlap                                   │
│                                                                                  │
└─────────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────┐
│ 2. HALLUCINATION DETECTION THRESHOLDS                                            │
├─────────────────────────────────────────────────────────────────────────────────┤
│                                                                                  │
│  Faithfulness Score  │ Interpretation          │ Action                         │
│  ─────────────────────────────────────────────────────────────────────────────  │
│  >= 0.80             │ Highly Faithful         │ Safe for production            │
│  0.60 - 0.80         │ Mostly Faithful         │ Review edge cases              │
│  0.40 - 0.60         │ Partially Faithful      │ Manual verification needed     │
│  0.20 - 0.40         │ Likely Hallucination    │ Flag for review                │
│  < 0.20              │ Severe Hallucination    │ Block from production          │
│                                                                                  │
└─────────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────┐
│ 3. MODEL SELECTION GUIDELINES                                                    │
├─────────────────────────────────────────────────────────────────────────────────┤
│                                                                                  │
│  Embedding Models (Semantic Similarity):                                         │
│  • all-MiniLM-L6-v2: Fast, good for general use                                 │
│  • all-mpnet-base-v2: Higher quality, slower                                    │
│  • paraphrase-multilingual: Multi-language support                              │
│                                                                                  │
│  NLI Models (Entailment):                                                        │
│  • cross-encoder/nli-deberta-v3-small: Good balance                             │
│  • cross-encoder/nli-deberta-v3-base: Higher quality                            │
│  • cross-encoder/nli-roberta-base: Alternative option                           │
│                                                                                  │
└─────────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────┐
│ 4. COMMON FAITHFULNESS PITFALLS                                                  │
├─────────────────────────────────────────────────────────────────────────────────┤
│                                                                                  │
│  ❌ Using only token overlap (misses semantic understanding)                     │
│  ❌ Not handling pandas Series in MLflow metric functions                        │
│  ❌ Ignoring claim-level analysis for long-form answers                          │
│  ❌ Using single threshold for all content types                                 │
│  ❌ Not verifying citations separately                                           │
│  ❌ Treating neutral NLI as unfaithful (it's uncertain, not wrong)              │
│                                                                                  │
└─────────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────┐
│ 5. IMPROVING RAG FAITHFULNESS                                                    │
├─────────────────────────────────────────────────────────────────────────────────┤
│                                                                                  │
│  Retrieval Improvements:                                                         │
│  • Use dense retrieval with semantic search                                      │
│  • Implement re-ranking for better context selection                             │
│  • Filter irrelevant chunks before generation                                    │
│                                                                                  │
│  Generation Improvements:                                                        │
│  • Add "Only use provided context" instructions                                 │
│  • Require inline citations                                                      │
│  • Use chain-of-thought for grounding                                           │
│  • Lower temperature for more deterministic outputs                              │
│                                                                                  │
│  Post-processing:                                                                │
│  • Implement answer-context verification                                         │
│  • Add confidence scores                                                         │
│  • Flag uncertain answers for human review                                       │
│                                                                                  │
└─────────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────┐
│ 6. FAITHFULNESS vs RELEVANCE vs CORRECTNESS                                      │
├─────────────────────────────────────────────────────────────────────────────────┤
│                                                                                  │
│  Metric        │ Question                          │ Example                     │
│  ─────────────────────────────────────────────────────────────────────────────  │
│  Faithfulness  │ Is answer supported by context?   │ "Paris" from doc → ✓       │
│  Relevance     │ Does answer address the question? │ Answers "capital?" → ✓     │
│  Correctness   │ Is the answer factually true?     │ Paris IS capital → ✓       │
│                                                                                  │
│  An answer can be:                                                               │
│  • Faithful but irrelevant (answers wrong question from context)                │
│  • Relevant but unfaithful (answers question but not from context)              │
│  • Faithful but incorrect (context itself has errors)                           │
│                                                                                  │
│  → Use all three metrics for comprehensive RAG evaluation                       │
│                                                                                  │
└─────────────────────────────────────────────────────────────────────────────────┘
"""

print(best_practices)


## 11. Summary and Next Steps


In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════════════════════╗
║                        SUMMARY: FAITHFULNESS WITH MLFLOW                         ║
╚══════════════════════════════════════════════════════════════════════════════════╝

✅ WHAT WE COVERED:

1. FAITHFULNESS FUNDAMENTALS
   • Semantic similarity-based faithfulness (embeddings)
   • NLI-based faithfulness (entailment detection)
   • Token overlap faithfulness (baseline)
   • Combined/comprehensive faithfulness scoring

2. HALLUCINATION DETECTION
   • Risk categorization (Low/Medium/High/Critical)
   • Claim-level verification
   • Indicator analysis (NLI contradiction, low similarity, etc.)

3. MLFLOW INTEGRATION
   • Custom metric creation with make_metric()
   • Proper handling of pandas Series inputs
   • Logging metrics, parameters, and artifacts
   • Generating evaluation reports and recommendations

4. ADVANCED USE CASES
   • Claim-level faithfulness (verify each statement)
   • Multi-context faithfulness (multiple retrieved documents)
   • Citation verification (inline citations matching sources)

5. PRODUCTION PIPELINE
   • Comprehensive evaluation with FaithfulnessPipeline class
   • Automated reporting and recommendations
   • Model comparison functionality

═══════════════════════════════════════════════════════════════════════════════════

📊 KEY METRICS CREATED:
   • semantic_faithfulness    - Embedding-based similarity
   • nli_faithfulness         - NLI entailment scoring
   • token_overlap_faithfulness - Token overlap baseline
   • comprehensive_faithfulness - Weighted combination (50% NLI + 35% semantic + 15% overlap)

🔍 HALLUCINATION THRESHOLDS:
   • >= 0.80: Highly Faithful (production ready)
   • 0.60-0.80: Mostly Faithful (monitor)
   • 0.40-0.60: Partially Faithful (review needed)
   • < 0.40: Potential Hallucination (flag/block)

🔗 NEXT STEPS:
   1. View results in MLflow UI: mlflow ui
   2. Compare experiments across runs
   3. Combine with relevance and correctness metrics
   4. Set up automated evaluation pipelines for RAG systems
   5. Implement human-in-the-loop for low-confidence answers

═══════════════════════════════════════════════════════════════════════════════════

📚 RELATED METRICS TO EXPLORE:
   • Answer Similarity - For evaluating answer quality vs reference
   • ROUGE/BLEU - For summarization and translation
   • Exact Match - For factoid QA
   • Relevance - For checking if answer addresses the question

═══════════════════════════════════════════════════════════════════════════════════
""")


In [ ]:
# Set up MLflow experiment
mlflow.set_experiment(f"/Users/{user_email}/faithfulness-demo")

def evaluate_rag_model(model_name: str,
                       predictions: List[str],
                       contexts: List[str],
                       model_config: Dict = None) -> Tuple[Dict, Dict]:
    """
    Evaluate a RAG model using faithfulness metrics and log to MLflow.
    """
    with mlflow.start_run(run_name=model_name):
        # Log model configuration
        mlflow.log_param("model_name", model_name)
        mlflow.log_param("num_samples", len(predictions))
        mlflow.log_param("embedding_model", faithfulness_metrics.embedding_model_name)
        mlflow.log_param("nli_model", faithfulness_metrics.nli_model_name)

        if model_config:
            for key, value in model_config.items():
                mlflow.log_param(key, value)

        # Create evaluation dataframe
        # Note: For faithfulness, we use context as targets
        eval_df = pd.DataFrame({
            "predictions": predictions,
            "targets": contexts  # Context is used as target for faithfulness
        })

        # Evaluate with faithfulness metrics
        metrics = [
            semantic_faithfulness,
            nli_faithfulness,
            token_overlap,
            comprehensive_faithfulness
        ]

        results = mlflow.evaluate(
            data=eval_df,
            targets="targets",
            predictions="predictions",
            extra_metrics=metrics,
            evaluators="default"
        )

        # Calculate additional detailed metrics
        detailed_scores = calculate_detailed_faithfulness(predictions, contexts)

        # Log detailed metrics
        for key, value in detailed_scores.items():
            if isinstance(value, (int, float)):
                mlflow.log_metric(key, value)

        return results.metrics, detailed_scores

def calculate_detailed_faithfulness(predictions: List[str], contexts: List[str]) -> Dict:
    """
    Calculate detailed faithfulness statistics.
    """
    semantic_scores = []
    nli_scores = []
    overlap_scores = []
    
    for pred, ctx in zip(predictions, contexts):
        semantic_scores.append(faithfulness_metrics.compute_semantic_faithfulness(pred, ctx))
        nli_scores.append(faithfulness_metrics.compute_nli_faithfulness(pred, ctx)['faithfulness'])
        overlap_scores.append(faithfulness_metrics.compute_token_overlap_faithfulness(pred, ctx))
    
    # Combined scores
    combined_scores = [0.5*n + 0.35*s + 0.15*o for n, s, o in zip(nli_scores, semantic_scores, overlap_scores)]
    
    detailed = {
        # Basic statistics
        "faithfulness_mean": np.mean(combined_scores),
        "faithfulness_std": np.std(combined_scores),
        "faithfulness_min": np.min(combined_scores),
        "faithfulness_max": np.max(combined_scores),
        "faithfulness_median": np.median(combined_scores),
        
        # Individual metric means
        "semantic_mean": np.mean(semantic_scores),
        "nli_mean": np.mean(nli_scores),
        "overlap_mean": np.mean(overlap_scores),
        
        # Quality distribution
        "high_faithfulness": sum(1 for s in combined_scores if s >= 0.8),
        "medium_faithfulness": sum(1 for s in combined_scores if 0.5 <= s < 0.8),
        "low_faithfulness": sum(1 for s in combined_scores if s < 0.5),
        
        # Percentages
        "high_pct": sum(1 for s in combined_scores if s >= 0.8) / len(combined_scores) * 100,
        "medium_pct": sum(1 for s in combined_scores if 0.5 <= s < 0.8) / len(combined_scores) * 100,
        "low_pct": sum(1 for s in combined_scores if s < 0.5) / len(combined_scores) * 100,
        
        # Hallucination detection
        "potential_hallucinations": sum(1 for s in combined_scores if s < 0.4)
    }
    
    return detailed

# Evaluate all three model variants
models_config = {
    "High-Faithfulness-RAG": {
        "predictions": df_rag["faithful_answer"].tolist(),
        "config": {"architecture": "RAG-GPT4", "retriever": "dense", "grounding": "strict"}
    },
    "Medium-Faithfulness-RAG": {
        "predictions": df_rag["partial_faithful"].tolist(),
        "config": {"architecture": "RAG-GPT3.5", "retriever": "sparse", "grounding": "moderate"}
    },
    "Low-Faithfulness-Baseline": {
        "predictions": df_rag["unfaithful_answer"].tolist(),
        "config": {"architecture": "GPT-baseline", "retriever": "none", "grounding": "none"}
    }
}

evaluation_results = {}
for model_name, config in models_config.items():
    print(f"\nEvaluating {model_name}...")
    metrics, detailed = evaluate_rag_model(
        model_name,
        config["predictions"],
        df_rag["context"].tolist(),
        config["config"]
    )
    evaluation_results[model_name] = {"metrics": metrics, "detailed": detailed}

print("\n✅ Evaluation completed for all models!")
